<a href="https://colab.research.google.com/github/saraanwer15/Brain_Tumour/blob/main/bhaiyya's_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import random

import numpy as np
import torch
from skimage.io import imread
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import argparse
from torch import Tensor

In [12]:
class BrainSegmentationDataset(Dataset):

    def __init__(self, image_files):
        self.image_files = image_files
        self.mask_files = []
        for filepath in self.image_files:
                mask_path = filepath.split(".")[-2] + "_mask." + filepath.split(".")[-1]
                self.mask_files.append(mask_path)

    def __getitem__(self,i):
      image_tensor = Tensor(imread(self.image_files[i]))
      mask_tensor = Tensor(imread(self.mask_files[i], as_gray=True))
      image_tensor = image_tensor.permute(2,0,1)*1/255.0
      #mask_tensor = mask_tensor.permute(2,0,1)*1/255.0
      tup = (image_tensor,mask_tensor)
      return tup
        
    def __len__(self):
      return len(self.image_files)


In [13]:
path = ["/content/drive/My Drive/archive/kaggle_3m/TCGA_CS_4941_19960909/TCGA_CS_4941_19960909_2.tif"]
a= BrainSegmentationDataset(path)

In [14]:
i=0
for (dirpath, dirnames, filenames) in os.walk("/content/drive/My Drive/archive/kaggle_3m"):
            for filename in sorted(
                filter(lambda f: ".tif" in f, filenames),
                key=lambda x: int(x.split(".")[-2].split("_")[4]),):
              i= i+1
print(i)

6208


In [15]:
def split_random(image_files, p):
  l1 = []
  l2 = []
  for i in range(0,len(image_files)):
    if np.random.uniform()<p:
      l1.append(image_files[i])     
    else:
      l2.append(image_files[i])
  return l1,l2

In [16]:
def datasets(image_files_train, image_files_valid):
    train = BrainSegmentationDataset(
        image_files=image_files_train,
    )
    valid = BrainSegmentationDataset(
        image_files=image_files_valid
    )
    return train, valid

In [17]:
def load_data(image_path):
  train_ds, valid_ds = split_random(image_path, 0.8)
  train_ds, valid_ds = datasets(train_ds, valid_ds)
  train_ld = DataLoader(train_ds, batch_size=32, shuffle=True)
  valid_ld = DataLoader(valid_ds, batch_size=32)
  return train_ld,valid_ld

In [18]:
device = torch.device("cpu" if not torch.cuda.is_available() else "cuda:0")
image_path = []
i=0
for (dirpath, dirnames, filenames) in os.walk("/content/drive/My Drive/archive/kaggle_3m"):
  for filename in sorted( filter(lambda f: ".tif" in f, filenames), key=lambda x: int(x.split(".")[-2].split("_")[4]),):
    filepath = os.path.join(dirpath, filename)
    if "mask" not in filename:
      i= i+1
      image_path.append(filepath)
loader_train, loader_valid = load_data(image_path)
loaders = {"train": loader_train, "valid": loader_valid}        
print(i)

3104


In [19]:
for i,data in enumerate(loader_valid):
  x,y=data
  print(x.shape, "abc", y.shape)

torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 256]) abc torch.Size([32, 256, 256])
torch.Size([32, 3, 256, 

In [20]:
type(loader_train)

torch.utils.data.dataloader.DataLoader

In [21]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [22]:
unet = UNet()
unet.to(device)


UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [25]:
import torch.optim as optim
 y_pred = torch.reshape(y_pred,(y_pred.shape[0],-1))
y_true = torch.reshape(y_true,(y_true.shape[0],-1))
dsc_loss = torch.nn.CrossEntropyLoss
best_validation_dsc = 0.0

optimizer = optim.Adam(unet.parameters(), lr=0.0001)

loss_train = []
loss_valid = []

step = 0

In [32]:
def dsc_per_volume(validation_pred, validation_true, patient_slice_index):
    dsc_list = []
    num_slices = np.bincount([p[0] for p in patient_slice_index])
    index = 0
    for p in range(len(num_slices)):
        y_pred = np.array(validation_pred[index : index + num_slices[p]])
        y_true = np.array(validation_true[index : index + num_slices[p]])
        dsc_list.append(dsc(y_pred, y_true))
        index += num_slices[p]
    return dsc_list

In [ ]:
from tqdm import tqdm
for epoch in tqdm(range(10), total=10):
        for phase in ["train", "valid"]:
            if phase == "train":
                unet.train()
            else:
                unet.eval()

            validation_pred = []
            validation_true = []

            for i, data in enumerate(loaders[phase]):
                if phase == "train":
                    step += 1

                x, y_true = data
                x, y_true = x.to(device), y_true.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "train"):
                    y_pred = unet(x)

                    loss = dsc_loss(y_pred, y_true)

                    if phase == "valid":
                        loss_valid.append(loss.item())
                        y_pred_np = y_pred.detach().cpu().numpy()
                        validation_pred.extend(
                            [y_pred_np[s] for s in range(y_pred_np.shape[0])]
                        )
                        y_true_np = y_true.detach().cpu().numpy()
                        validation_true.extend(
                            [y_true_np[s] for s in range(y_true_np.shape[0])]
                        )

                    if phase == "train":
                        loss_train.append(loss.item())
                        loss.backward()
                        optimizer.step()


            if (epoch + 1) % 10 == 0:
              print('Epoch [{}/{}] - Iter[{}/{}], MSE loss:{:.6f}'.format(
                epoch + 1, num_epoch, i + 1,
                len(train_loader.dataset) // 32, loss.item() / 32
            ))






  0%|          | 0/10 [00:00<?, ?it/s]




 10%|█         | 1/10 [00:54<08:08, 54.24s/it]




 20%|██        | 2/10 [01:49<07:16, 54.60s/it]




 30%|███       | 3/10 [02:45<06:24, 54.96s/it]




 40%|████      | 4/10 [03:41<05:32, 55.38s/it]




 50%|█████     | 5/10 [04:38<04:38, 55.79s/it]